In [51]:
import pandas as pd

data = pd.read_csv('6_main_orders.csv')
print(data.head())
original_count = len(data)
print(f"Original Rows: {original_count}")

# 1. Drop Null
data = data.dropna()
print(f"After Dropping NA Rows: {len(data)}")

# 2. Drop Duplicates
data = data.drop_duplicates(subset='order_id')
print(f"After dropping duplicate rows: {len(data)}")

# 3. Convert Type + handling mistaken date: Convert `order_date` to datetime format
data['order_date'] = pd.to_datetime(data['order_date'], errors='coerce')
# drop converting failed rows
data = data.dropna(subset=['order_date'])
print(f"After converting the data: {len(data)}")

# 4. Remove Outliers: Delete rows where `quantity` ≤ 0 or > 10, `unit_price` ≤ 0, or `customer_rating` < 1 or > 5
data = data[
    (data['quantity'] > 0) & 
    (data['quantity'] <= 10) & 
    (data['unit_price'] > 0) & 
    (data['customer_rating'].between(1, 5))
]
print(f"Remove outlier rows: {len(data)}")

# 5. Rename Columns: Rename `total_amount` to `revenue`
data.rename(columns={'total_amount': 'revenue'}, inplace=True)
print(f"After renaming columns: {data.columns.tolist()}")

# 6. Reorder Columns: Move `order_id`, `customer_id`, `order_date` to the front
reorder_columns = ['order_id', 'customer_id', 'order_date']
other_cols = [col for col in data.columns if col not in reorder_columns]
data = data[reorder_columns + other_cols]

# 7. Filter Rows: Keep only orders where `product_category` is "Electronics" AND `revenue` > 500
data = data[
    (data['product_category'] == 'Electronics') & 
    (data['revenue'] > 500)
]

# 8. Sort Data: Sort by `revenue` in descending order
data = data.sort_values('revenue', ascending=False)

# 9. Add New Columns: Add `is_high_value` column (True if revenue > 1000)
data['is_high_value'] = data['revenue'] > 1000
cleaned_count = len(data)

# 10. Summary
print(f"Final Rows: {len(data)}")
print(f"Deleted rows: {original_count - len(data)} ")

  order_id customer_id   customer_name  order_date product_category  quantity  \
0   ORD001        C001     Alice Smith  2024-01-15      Electronics       2.0   
1   ORD002        C002     Bob Johnson  2024-01-16         Clothing       3.0   
2   ORD003        C003  Carol Williams  2024-01-17             Food       5.0   
3   ORD004        C004     David Brown  2024-01-18      Electronics       1.0   
4   ORD005        C005      Emma Davis  2024-01-19         Clothing       4.0   

   unit_price  total_amount payment_method  customer_rating  discount_applied  \
0      899.99       1799.98    Credit Card              4.5              10.0   
1       59.99        179.97         PayPal              4.0               5.0   
2       12.99         64.95           Cash              3.5               0.0   
3     1299.99       1299.99    Credit Card              5.0              15.0   
4       39.99        159.96         PayPal              3.0               8.0   

  shipping_city shipping_c

In [52]:
customer_data = pd.read_csv('6_customer_info.csv')
customer_data = customer_data.dropna()
customer_data = customer_data.drop_duplicates()
# 11. Merge: Merge `main_orders.csv` and `customer_info.csv` on `customer_id`
merged_data = pd.merge(data, customer_data, on='customer_id', how='inner', suffixes=('', '_cus'))
print(merged_data.head())
# 12. Filter: Keep only orders where `customer_tier` is "Gold" AND `age` is between 25-45
merged_data = merged_data[(merged_data['customer_tier']=='Gold') & (merged_data['age'].between(25, 45))]
# 13. Add Column: Add `profit_margin` column (assume cost is 70% of revenue)
merged_data['profit_margin'] = merged_data['revenue'] * 0.3
# 14. Group & Aggregate: Group by `customer_tier`, calculate average revenue, average age, and order count
analysis = merged_data.groupby('customer_tier', observed=False).agg(
    avg_revenue=('revenue', 'mean'),
    avg_age=('age', 'mean'),
    total_orders=('order_id', 'count')
    ).reset_index()
analysis = analysis.sort_values('avg_revenue', ascending=False)
final = merged_data[['customer_name', 'age', 'customer_tier', 'revenue', 'profit_margin']].sort_values('revenue', ascending=False)
final_count = len(final)
final.to_csv('6_homework.csv', index=False)
print(f"Original Rows: {original_count}")
print(f"After Cleaning: {cleaned_count}")
print(f"Final Rows: {final_count}")

               order_id customer_id order_date customer_name product_category  \
0                ORD001        C001 2024-01-15   Alice Smith      Electronics   
1                ORD101        C001 2024-04-25   Alice Smith      Electronics   
2  ---DUPLICATED_ROW---        C001 2024-01-15   Alice Smith      Electronics   
3                ORD022        C022 2024-02-05     Uma Lewis      Electronics   
4                ORD004        C004 2024-01-18   David Brown      Electronics   

   quantity  unit_price  revenue payment_method  customer_rating  ...  \
0       2.0      899.99  1799.98    Credit Card              4.5  ...   
1       2.0      899.99  1799.98    Credit Card              4.5  ...   
2       2.0      899.99  1799.98    Credit Card              4.5  ...   
3       2.0      799.99  1599.98    Credit Card              4.5  ...   
4       1.0     1299.99  1299.99    Credit Card              5.0  ...   

   shipping_country is_high_value customer_name_cus                  email